In [0]:
pip install azure-storage-blob

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
!pip install pandas openpyxl

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
from azure.storage.blob import BlobServiceClient
# storage_account_name="insurancedataset2025"
# storage_account_key="eIOZmD5913OT0f5DqbBQBKiu+ghZwqAPvt2s+7wYAcRZTdvjUcTGLyYODcYNxG9WL9pg1+tZB4ht+ASte4zRLg=="
connection_string="DefaultEndpointsProtocol=https;AccountName=insurancedataset2025;AccountKey=eIOZmD5913OT0f5DqbBQBKiu+ghZwqAPvt2s+7wYAcRZTdvjUcTGLyYODcYNxG9WL9pg1+tZB4ht+ASte4zRLg==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
for i in blob_service_client.list_containers():
  print(i.name)


bronze
destination
gold
logs
silver
source


In [0]:
new_containers=["bronze","silver","gold",'logs']
for i in new_containers:
    try :
        blob_service_client.create_container(i)
        print(i)
    except:
        print("already exists")

already exists
already exists
already exists
already exists


In [0]:
container_name='source'
container_client=blob_service_client.get_container_client(container_name)
for i in container_client.list_blobs():
  print(i.name)

Retail Dataset.csv
Sample - Superstore (2).csv
Sample - Superstore1.csv
sample_submission.csv
store.csv
test.csv
train.csv


In [0]:
import pandas  as pd
from io import BytesIO
container_name="source"
file_name="Sample - Superstore1.csv"
blob_client=blob_service_client.get_blob_client(container=container_name,blob=file_name)
download_stream=blob_client.download_blob().readall()
df=pd.read_csv(BytesIO(download_stream),encoding="ISO-8859-1")
df


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,Florida,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


In [0]:
logs_container='logs'
df.head(2)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [0]:
import datetime
validations=[]
today=datetime.datetime.now().strftime("%Y-%m-%d")
today

def log_result(rule,status,details=""):
    validations.append({"Date":today,"Rule":rule,"status":status,"Details":details})  
log_result("File Loaded", "Pass" if not df.empty else "FAIL",f"rows:{len(df)}")
null_values_status="Pass" if df.isnull().sum().sum()==0 else "FAIL"
log_result("Null values",null_values_status,f"rows:{df.isnull().sum().sum()}")
log_result("Sales Range check", "True" if sales_range>0 else "False", sales_range)



---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4637446614165684>, line 11
      9 null_values_status="Pass" if df.isnull().sum().sum()==0 else "FAIL"
     10 log_result("Null values",null_values_status,f"rows:{df.isnull().sum().sum()}")
---> 11 log_result("Sales Range check", "True" if sales_range>0 else "False", sales_range)

NameError: name 'sales_range' is not defined

In [0]:
sales_range=(df[df['Sales']>3000].shape[0]/df.shape[0])*100

In [0]:
df.isna().sum().sum()

0

In [0]:
validations

In [0]:
validations_df=pd.DataFrame(validations)
validations_df
logs_blob_clinet=blob_service_client.get_blob_client(container=logs_container,blob="validations.csv")
logs_blob_clinet.upload_blob(validations_df.to_csv(index=False),overwrite=True)

{'etag': '"0x8DDFCF1C35CF218"',
 'last_modified': datetime.datetime(2025, 9, 26, 11, 42, 25, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'r\x90\xa9\xd7dC+ms(4\xd1\xf6\xfe_\xc5'),
 'client_request_id': 'df10f30c-9acd-11f0-8232-0affe9e0c937',
 'request_id': '7abac01b-a01e-00b3-14da-2ef325000000',
 'version': '2024-11-04',
 'version_id': None,
 'date': datetime.datetime(2025, 9, 26, 11, 42, 25, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None}

In [0]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import BytesIO
import datetime

# Azure connection
connection_string = "DefaultEndpointsProtocol=https;AccountName=insurancedataset2025;AccountKey=eIOZmD5913OT0f5DqbBQBKiu+ghZwqAPvt2s+7wYAcRZTdvjUcTGLyYODcYNxG9WL9pg1+tZB4ht+ASte4zRLg==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Source file details
source_container = "source"
file_name = "Sample - Superstore1.csv"

# Logs container (make sure 'logs' container already exists)
logs_container = "logs"

# Download the source CSV
blob_client = blob_service_client.get_blob_client(container=source_container, blob=file_name)
download_stream = blob_client.download_blob().readall()

# Read CSV with fallback encoding
try:
    df = pd.read_csv(BytesIO(download_stream), encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(BytesIO(download_stream), encoding="ISO-8859-1")

# --- Data Validations ---
validations = []
today = datetime.datetime.now().strftime("%Y-%m-%d")

def log_result(rule, status, details=""):
    validations.append({"Date": today, "Rule": rule, "Status": status, "Details": details})

# (20 checks as before)
log_result("File Loaded", "PASS" if not df.empty else "FAIL", f"Rows: {len(df)}")
log_result("Duplicate Rows", "FAIL" if df.duplicated().any() else "PASS")
log_result("Null Order ID", "FAIL" if df['Order ID'].isnull().any() else "PASS")
log_result("Null Order Date", "FAIL" if df['Order Date'].isnull().any() else "PASS")

try:
    pd.to_datetime(df['Order Date']); log_result("Valid Order Dates", "PASS")
except Exception as e: log_result("Valid Order Dates", "FAIL", str(e))

try:
    pd.to_datetime(df['Ship Date']); log_result("Valid Ship Dates", "PASS")
except Exception as e: log_result("Valid Ship Dates", "FAIL", str(e))

try:
    if (pd.to_datetime(df['Ship Date']) >= pd.to_datetime(df['Order Date'])).all():
        log_result("Ship Date after Order Date", "PASS")
    else:
        log_result("Ship Date after Order Date", "FAIL")
except: log_result("Ship Date after Order Date", "FAIL")

log_result("Negative Sales", "FAIL" if (df['Sales'] < 0).any() else "PASS")
log_result("Negative Quantity", "FAIL" if (df['Quantity'] < 0).any() else "PASS")
log_result("Discount Range", "FAIL" if not df['Discount'].between(0, 1).all() else "PASS")
log_result("Profit Null", "FAIL" if df['Profit'].isnull().any() else "PASS")
log_result("Order+Product Unique", "FAIL" if df.duplicated(['Order ID','Product ID']).any() else "PASS")
log_result("Future Order Dates", "FAIL" if (pd.to_datetime(df['Order Date']) > pd.Timestamp.today()).any() else "PASS")

if 'Postal Code' in df.columns:
    log_result("Postal Code Numeric", "FAIL" if not df['Postal Code'].astype(str).str.isnumeric().all() else "PASS")

if 'Country' in df.columns:
    log_result("Country United States", "FAIL" if not (df['Country'] == "United States").all() else "PASS")

if 'Category' in df.columns:
    valid_categories = {"Furniture", "Office Supplies", "Technology"}
    log_result("Category Valid", "FAIL" if not df['Category'].isin(valid_categories).all() else "PASS")

log_result("Row Count > 1000", "PASS" if len(df) > 1000 else "FAIL")
log_result("Sales > 0 if Quantity > 0", "FAIL" if ((df['Quantity'] > 0) & (df['Sales'] <= 0)).any() else "PASS")
log_result("Null Customer Name", "FAIL" if df['Customer Name'].isnull().any() else "PASS")

if 'State' in df.columns:
    log_result("Null State", "FAIL" if df['State'].isnull().any() else "PASS")

# --- Save log to CSV in-memory ---
log_df = pd.DataFrame(validations)
log_file = f"validation_log_{today}.csv"

csv_buffer = BytesIO()
log_df.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)

# --- Upload log CSV into logs container ---
logs_blob_client = blob_service_client.get_blob_client(container=logs_container, blob=log_file)
logs_blob_client.upload_blob(csv_buffer, overwrite=True)

print(f"✅ Validation log uploaded to container '{logs_container}' as '{log_file}'")


✅ Validation log uploaded to container 'logs' as 'validation_log_2025-09-26.csv'


In [0]:
### checking should be part of bronze
### creating ( Trasformations)
### order_date : yyyy:mm:dd , year,month, day, leap year, quarter, half, start of the month , end of the month

In [0]:
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

In [0]:
### df, null_value_check , outliers check, duplicate values check 
### net_sales= sales-(sales*discount)
### profit margins : (profit/sales)*100
### cls(customer life time sales) 
# (select customer_id,sum(sales) from table group by customer_id), df.groupby("customer_id").sum()['sales]
### customer_segmentation : 
        def customer_segment(sales):
            if sales<1000:
                return "low value customer"
            elif sales>=1000 and sales<5000:
                return "mid value customer"
            else:
                return "high value customer customer"
    df['sales'].apply(lambda row: customer_segment(row))

    


In [0]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import BytesIO
import datetime

# 🔹 Azure connection
connection_string = "DefaultEndpointsProtocol=https;AccountName=insurancedataset2025;AccountKey=eIOZmD5913OT0f5DqbBQBKiu+ghZwqAPvt2s+7wYAcRZTdvjUcTGLyYODcYNxG9WL9pg1+tZB4ht+ASte4zRLg==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# 🔹 Containers
source_container = "source"
bronze_container = "silver"
file_name = "Sample - Superstore1.csv"

# --- Step 1: Read Source File ---
blob_client = blob_service_client.get_blob_client(container=source_container, blob=file_name)
download_stream = blob_client.download_blob().readall()

try:
    df = pd.read_csv(BytesIO(download_stream), encoding="utf-8")
except UnicodeDecodeError:
    df = pd.read_csv(BytesIO(download_stream), encoding="ISO-8859-1")

# --- Step 2: Apply 30 Transformations ---

# 1. Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# 2. Remove duplicates
df.drop_duplicates(inplace=True)

# 3. Trim spaces
for col in df.select_dtypes(include="object"):
    df[col] = df[col].astype(str).str.strip()

# 4-5. Convert dates
df['order_date'] = pd.to_datetime(df['order_date'], errors="coerce")
df['ship_date'] = pd.to_datetime(df['ship_date'], errors="coerce")

# 6-11. Extract date parts
df['order_year'] = df['order_date'].dt.year
df['order_month'] = df['order_date'].dt.month
df['order_day'] = df['order_date'].dt.day
df['ship_year'] = df['ship_date'].dt.year
df['ship_month'] = df['ship_date'].dt.month
df['ship_day'] = df['ship_date'].dt.day

# 12. Shipping days
df['shipping_days'] = (df['ship_date'] - df['order_date']).dt.days

# 13. Standardize customer name
df['customer_name'] = df['customer_name'].str.title()

# 14. Robust split into first_name & last_name
df[['first_name', 'last_name']] = df['customer_name'].str.split(" ", n=1, expand=True)
df['last_name'] = df['last_name'].fillna("")

# 15. Standardize state
df['state'] = df['state'].str.title()

# 16. Postal Code 5 digits
df['postal_code'] = df['postal_code'].astype(str).str.zfill(5)

# 17. Country fixed
df['country'] = "United States"

# 18. Normalize Category
df['category'] = df['category'].replace({
    "furniture": "Furniture",
    "technology": "Technology",
    "office supplies": "Office Supplies"
})

# 19. Profit margin
df['profit_margin'] = df['profit'] / df['sales']

# 20. Sales per quantity
df['sales_per_quantity'] = df['sales'] / df['quantity']

# 21. High discount flag
df['high_discount'] = df['discount'] > 0.5

# 22. Negative profit flag
df['negative_profit'] = df['profit'] < 0

# 23. Region-State
df['region_state'] = df['region'] + "-" + df['state']

# 24. Order priority (rule based)
df['order_priority'] = df.apply(lambda x: "High" if x['discount'] > 0.4 else "Normal", axis=1)

# 25. Replace null postal codes
df['postal_code'] = df['postal_code'].replace("nan", "00000")

# 26. Cap discount at 0.8
df['discount'] = df['discount'].clip(upper=0.8)

# 27. Round sales/profit
df['sales'] = df['sales'].round(2)
df['profit'] = df['profit'].round(2)

# 28. Sales bucket
df['sales_bucket'] = pd.cut(df['sales'], bins=[0, 100, 500, df['sales'].max()], labels=["Low", "Medium", "High"])

# 29. Month name
df['order_month_name'] = df['order_date'].dt.strftime("%B")

# 30. Reorder columns
columns_order = [
    'order_id','order_date','order_year','order_month','order_day',
    'ship_date','ship_year','ship_month','ship_day','shipping_days',
    'customer_id','customer_name','first_name','last_name','segment',
    'country','state','region','region_state','postal_code','category',
    'sub_category','product_id','product_name','sales','profit','discount',
    'quantity','profit_margin','sales_per_quantity','high_discount',
    'negative_profit','order_priority','sales_bucket','order_month_name'
]
df = df[[col for col in columns_order if col in df.columns]]

# --- Step 3: Save Transformed File to Bronze ---
output_file = f"superstore_transformed_{datetime.datetime.now().strftime('%Y-%m-%d')}.csv"

csv_buffer = BytesIO()
df.to_csv(csv_buffer, index=False)
csv_buffer.seek(0)

bronze_blob_client = blob_service_client.get_blob_client(container=bronze_container, blob=output_file)
bronze_blob_client.upload_blob(csv_buffer, overwrite=True)

print(f"✅ Transformed Silver file saved to Bronze as '{output_file}'")


✅ Transformed Silver file saved to Bronze as 'superstore_transformed_2025-09-26.csv'


In [0]:
### Bronze - Silver
### Bronze(20 columns)- Feature engineering(40 columns) ( Derived columns, which is a combination of your general trasformations plus 
###business rules)

In [0]:
### log,sin ,tan cos,1/x

In [0]:
### silver to gold 
### gold is the final output

In [0]:
### if you are building gold layer for single project with single table
### if you are building gold layer for multiple projects with multiple tables with star or snowflake schemas

In [0]:
df.columns

Index(['order_id', 'order_date', 'order_year', 'order_month', 'order_day',
       'ship_date', 'ship_year', 'ship_month', 'ship_day', 'shipping_days',
       'customer_id', 'customer_name', 'first_name', 'last_name', 'segment',
       'country', 'state', 'region', 'region_state', 'postal_code', 'category',
       'product_id', 'product_name', 'sales', 'profit', 'discount', 'quantity',
       'profit_margin', 'sales_per_quantity', 'high_discount',
       'negative_profit', 'order_priority', 'sales_bucket',
       'order_month_name'],
      dtype='object')

In [0]:
dim_product=df[['product_id','product_name','category']].drop_duplicates()
date_dimension=df[['order_date','ship_date']].drop_duplicates()
customer_dimesion=df[['customer_id','customer_name','first_name','last_name','segment','country','state','region','region_state','postal_code']].drop_duplicates()

In [0]:
### Aggregations

In [0]:
df.groupby("category")['sales'].sum().reset_index()

,category,sales
0,Furniture,741999.73
1,Office Supplies,719046.90
2,Technology,836154.02


In [0]:
df.groupby("category")['profit'].sum().reset_index()

,category,profit
0,Furniture,18451.10
1,Office Supplies,122490.00
2,Technology,145455.44


In [0]:
df.groupby(["category","region"])['profit'].sum().reset_index()

,category,region,profit
0,Furniture,Central,-2871.07
1,Furniture,East,3046.13
2,Furniture,South,6771.18
3,Furniture,West,11504.86
4,Office Supplies,Central,8879.76
5,Office Supplies,East,41014.33
6,Office Supplies,South,19986.37
7,Office Supplies,West,52609.54
8,Technology,Central,33697.55
9,Technology,East,47462.04


In [0]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
from io import BytesIO
import datetime

# 🔹 Azure connection
connection_string = "DefaultEndpointsProtocol=https;AccountName=insurancedataset2025;AccountKey=eIOZmD5913OT0f5DqbBQBKiu+ghZwqAPvt2s+7wYAcRZTdvjUcTGLyYODcYNxG9WL9pg1+tZB4ht+ASte4zRLg==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

# Containers
silver_container = "silver"   # silver-transformed file stored in bronze
gold_container = "gold"

# File details
today = datetime.datetime.now().strftime("%Y-%m-%d")
silver_file = f"superstore_transformed_{today}.csv"

# --- Step 1: Read Silver File ---
silver_blob_client = blob_service_client.get_blob_client(container=silver_container, blob=silver_file)
download_stream = silver_blob_client.download_blob().readall()
df = pd.read_csv(BytesIO(download_stream))

# --- Step 2: Fact Table ---
fact_sales_cols = [c for c in [
    "order_id","product_id","customer_id","order_date","order_year","order_month",
    "quantity","sales","discount","profit","profit_margin","shipping_days"
] if c in df.columns]
fact_sales = df[fact_sales_cols]

# --- Step 3: Dimension Tables (Safe Selection) ---
customer_cols = [c for c in ["customer_id","customer_name","first_name","last_name","segment","region","state","postal_code"] if c in df.columns]
dim_customer = df[customer_cols].drop_duplicates()

product_cols = [c for c in ["product_id","product_name","sub_category","category"] if c in df.columns]
dim_product = df[product_cols].drop_duplicates()

date_cols = [c for c in ["order_date","order_year","order_month","order_day","order_month_name"] if c in df.columns]
dim_date = df[date_cols].drop_duplicates()
if "order_date" in dim_date.columns:
    dim_date["date_key"] = pd.factorize(dim_date["order_date"])[0] + 1

region_cols = [c for c in ["region","state","country"] if c in df.columns]
dim_region = df[region_cols].drop_duplicates()

# --- Step 4: 20 Aggregations ---
aggregations = {
    "sales_by_category": df.groupby("category")["sales"].sum().reset_index() if "category" in df.columns else pd.DataFrame(),
    "profit_by_category": df.groupby("category")["profit"].sum().reset_index() if "category" in df.columns else pd.DataFrame(),
    "sales_by_subcategory": df.groupby("sub_category")["sales"].sum().reset_index() if "sub_category" in df.columns else pd.DataFrame(),
    "avg_discount_by_category": df.groupby("category")["discount"].mean().reset_index() if "category" in df.columns else pd.DataFrame(),
    "avg_margin_by_category": df.groupby("category")["profit_margin"].mean().reset_index() if "category" in df.columns else pd.DataFrame(),
    "sales_by_segment": df.groupby("segment")["sales"].sum().reset_index() if "segment" in df.columns else pd.DataFrame(),
    "profit_by_segment": df.groupby("segment")["profit"].sum().reset_index() if "segment" in df.columns else pd.DataFrame(),
    "sales_by_region": df.groupby("region")["sales"].sum().reset_index() if "region" in df.columns else pd.DataFrame(),
    "profit_by_region": df.groupby("region")["profit"].sum().reset_index() if "region" in df.columns else pd.DataFrame(),
    "sales_by_state": df.groupby("state")["sales"].sum().reset_index() if "state" in df.columns else pd.DataFrame(),
    "top10_products_sales": df.groupby("product_name")["sales"].sum().reset_index().nlargest(10, "sales") if "product_name" in df.columns else pd.DataFrame(),
    "top10_products_profit": df.groupby("product_name")["profit"].sum().reset_index().nlargest(10, "profit") if "product_name" in df.columns else pd.DataFrame(),
    "monthly_sales": df.groupby(["order_year","order_month"])["sales"].sum().reset_index() if {"order_year","order_month"}.issubset(df.columns) else pd.DataFrame(),
    "monthly_profit": df.groupby(["order_year","order_month"])["profit"].sum().reset_index() if {"order_year","order_month"}.issubset(df.columns) else pd.DataFrame(),
    "yearly_sales": df.groupby("order_year")["sales"].sum().reset_index() if "order_year" in df.columns else pd.DataFrame(),
    "sales_segment_region": df.groupby(["segment","region"])["sales"].sum().reset_index() if {"segment","region"}.issubset(df.columns) else pd.DataFrame(),
    "profit_category_region": df.groupby(["category","region"])["profit"].sum().reset_index() if {"category","region"}.issubset(df.columns) else pd.DataFrame(),
    "avg_shipping_days_region": df.groupby("region")["shipping_days"].mean().reset_index() if {"region","shipping_days"}.issubset(df.columns) else pd.DataFrame(),
    "sales_per_customer": df.groupby("customer_id")["sales"].sum().reset_index() if "customer_id" in df.columns else pd.DataFrame(),
    "profit_per_customer": df.groupby("customer_id")["profit"].sum().reset_index() if "customer_id" in df.columns else pd.DataFrame()
}

# --- Step 5: Upload Fact & Dimension Tables to Gold ---
tables = {
    "fact_sales": fact_sales,
    "dim_customer": dim_customer,
    "dim_product": dim_product,
    "dim_date": dim_date,
    "dim_region": dim_region
}

for name, tdf in tables.items():
    if not tdf.empty:
        csv_buffer = BytesIO()
        tdf.to_csv(csv_buffer, index=False)
        csv_buffer.seek(0)
        blob_name = f"facts_dimensions/{name}_{today}.csv"
        gold_blob_client = blob_service_client.get_blob_client(container=gold_container, blob=blob_name)
        gold_blob_client.upload_blob(csv_buffer, overwrite=True)
        print(f"✅ Table saved: {blob_name}")
    else:
        print(f"⚠️ Skipped {name}, no columns found")

# --- Step 6: Upload Aggregations to Gold ---
for name, agg_df in aggregations.items():
    if not agg_df.empty:
        csv_buffer = BytesIO()
        agg_df.to_csv(csv_buffer, index=False)
        csv_buffer.seek(0)
        blob_name = f"aggregations/{name}_{today}.csv"
        gold_blob_client = blob_service_client.get_blob_client(container=gold_container, blob=blob_name)
        gold_blob_client.upload_blob(csv_buffer, overwrite=True)
        print(f"✅ Aggregation saved: {blob_name}")
    else:
        print(f"⚠️ Skipped {name}, missing required columns")


✅ Table saved: facts_dimensions/fact_sales_2025-09-26.csv
✅ Table saved: facts_dimensions/dim_customer_2025-09-26.csv
✅ Table saved: facts_dimensions/dim_product_2025-09-26.csv
✅ Table saved: facts_dimensions/dim_date_2025-09-26.csv
✅ Table saved: facts_dimensions/dim_region_2025-09-26.csv
✅ Aggregation saved: aggregations/sales_by_category_2025-09-26.csv
✅ Aggregation saved: aggregations/profit_by_category_2025-09-26.csv
⚠️ Skipped sales_by_subcategory, missing required columns
✅ Aggregation saved: aggregations/avg_discount_by_category_2025-09-26.csv
✅ Aggregation saved: aggregations/avg_margin_by_category_2025-09-26.csv
✅ Aggregation saved: aggregations/sales_by_segment_2025-09-26.csv
✅ Aggregation saved: aggregations/profit_by_segment_2025-09-26.csv
✅ Aggregation saved: aggregations/sales_by_region_2025-09-26.csv
✅ Aggregation saved: aggregations/profit_by_region_2025-09-26.csv
✅ Aggregation saved: aggregations/sales_by_state_2025-09-26.csv
✅ Aggregation saved: aggregations/top10_pr